In [3]:
import ipywidgets as widgets
from IPython.display import display
import requests
from natsort import natsorted


class AggregateSelectorDashboard:
    def __init__(self, user, host, output):
        self.user = user
        self.host = host
        self.out = output
        self.global_aggregates = widgets.Button(
            description='View Global Aggregates',
            disabled=False,
            button_style='success', # 'success', 'info', 'warning', 'danger' or ''
            tooltip='Click here to view global aggregates',
            icon='check', # (FontAwesome names without the `fa-` prefix)
            layout=widgets.Layout(width='40%')
        )
        self.chain_aggregates = widgets.Button(
            description='View Local Aggregates',
            disabled=False,
            button_style='success', # 'success', 'info', 'warning', 'danger' or ''
            tooltip='Click here to view local aggregates',
            icon='check', # (FontAwesome names without the `fa-` prefix)
            layout=widgets.Layout(width='40%')
        )
        self.hotel_aggregates = widgets.Button(
            description='View Local Aggregates',
            disabled=False,
            button_style='success', # 'success', 'info', 'warning', 'danger' or ''
            tooltip='Click here to view hotel aggregates',
            icon='check', # (FontAwesome names without the `fa-` prefix)
            layout=widgets.Layout(width='40%')
        )
        hotels = requests.get(f"{host}/deceptecons/hotel").json()
        hotels = list(map(lambda x: f"{x['hid']}: {x['hname']}", filter(lambda x: x['hid'] != -1,hotels)))
        natsorted(hotels)
        self.hotel_options =widgets.Dropdown (
            placeholder='Choose a Hotel',
            options=hotels,
            description='Hotels:',
            disabled=False,
            layout=widgets.Layout(width='40%')
        )
        
        hotels = requests.get(f"{host}/deceptecons/hotel").json()
        supervisorChid = list(map(lambda x: x['chid'], filter(lambda x: x['hid'] != self.user['hid'], hotels)))[0]
        supervisorHotels = list(map(lambda x: [f"{x['hid']}: {x['hname']}", x['hid']],filter(lambda x: x['chid'] == supervisorChid,hotels)))
        self.hotel_supervisor_options =widgets.Dropdown (
            placeholder='Choose a Hotel',
            options=supervisorHotels,
            description='Hotels:',
            disabled=False,
            layout=widgets.Layout(width='40%')
        )
        self.tables_management_dashboard = widgets.Button(
            description='Manage Tables',
            disabled=False,
            button_style='success', # 'success', 'info', 'warning', 'danger' or ''
            tooltip='Click here to manage tables',
            icon='check', # (FontAwesome names without the `fa-` prefix)
            layout=widgets.Layout(width='40%')
        )
        self.return_to_dashboard_button = widgets.Button(
            description='Return to Aggregates Selection Dashboard',
            disabled=False,
            button_style='info', # 'success', 'info', 'warning', 'danger' or ''
            tooltip='Click me',
            icon='check', # (FontAwesome names without the `fa-` prefix)
            layout=widgets.Layout(width='40%')
        )
        
        self.global_aggregates.on_click(self.displayGlobalAggregates)
        self.chain_aggregates.on_click(self.displayLocalAggregates)
        self.hotel_aggregates.on_click(self.displayHotelAggregates)
        self.tables_management_dashboard.on_click(self.displayTableManagement)
        self.return_to_dashboard_button.on_click(self.return_to_dashboard)

    def return_to_dashboard(self,b ):
        self.out.clear_output()
        with self.out:
            %run AggregateSelectorDasboard.ipynb
            run(self.user, self.host, self.out)

    def displayGlobalAggregates(self,b):
        with self.out:
            self.hideAggregateSelectorDashboard()
            %run GlobalAggregatesDashboard.ipynb
            run(self.user, self.host, self.out)

    def displayLocalAggregates(self,b):
        with self.out:
            self.hideAggregateSelectorDashboard()
            self.user['hid'] = self.hotel_supervisor_options.value
            %run HotelAggregatesDashboard.ipynb
            run(self.user, self.host, self.out)
        

    def displayHotelAggregates(self,b):
        if self.user['hid'] == -1:
            self.user['hid'] = self.hotel_options.value.split(":")[0]
        self.hideAggregateSelectorDashboard()
        with self.out:
            %run HotelAggregatesDashboard.ipynb
            run(self.user, self.host, self.out)

    def displayTableManagement(self,b):
        with self.out:
            self.hideAggregateSelectorDashboard()
            %run TableManagementDashboard.ipynb
            run(self.user, self.host, self.out)
    
    def displayAggregateSelectorDashboard(self):
        with self.out:
            display(self.return_to_dashboard_button)
            if self.user['position'] == 'Administrator':
                display(self.global_aggregates)
                display(widgets.HBox([self.hotel_aggregates, self.hotel_options]))
            elif self.user['position'] == 'Supervisor':
                display(widgets.HBox([self.chain_aggregates, self.hotel_supervisor_options]))
            else:
                display(widgets.HBox([self.hotel_aggregates]))
            display(self.tables_management_dashboard)

    def hideAggregateSelectorDashboard(self):
        self.global_aggregates.close()
        self.chain_aggregates.close()
        self.hotel_aggregates.close()
        self.hotel_options.close()
        self.hotel_supervisor_options.close()
        self.tables_management_dashboard.close()


def run(user, host, output):
    x = AggregateSelectorDashboard(user, host, out)
    with x.out:
        x.displayAggregateSelectorDashboard()
    x.out